# ACS AWS Training: Day 2
### Nathan Miles
06/04/2020

## Outline
- Review of last time

- Some more initialization steps
    - Configuring the credentials file
        
- How do we keep people from logging into our EC2 instance?
     - SSH keys
    - Security Groups 
- Launching an EC2 instance
- Example
    - Downloading HST data from the MAST public cache in S3 to EC2
    
    

## Review From Day 1
- AWS is a giant web API
    - AWS Management Console: browser interface (web application built on the web API)
    - AWS Command Line Interface (CLI): exactly what it sounds like
    - Boto3: AWS Python Software Development Kit (SDK)
- IAM: Identity and Access Mangement
    - Authentication and authorization
- S3: Simple Storage Service
    - Storage and static-website hosting
- EC2: Elastic Cloud Compute
    - Servers in the cloud
- Lambda
    - On-demand compute resources without ever provisioning servers
- All services on AWS is pay-as-you-go and pay-per-use

## Programmatic AWS Access
- In order to access AWS via AWS CLI or Boto3 you must create a credentials file
    - These keys are used to authenticate requests submitted programmatically
    
- This even includes when you are working on EC2 instances

- These keys should be provide the holder with the same level of access to your AWS account with having to know anything about your username or password

- When generating keys for your personal account, please adhere to the best practices guide
    - <a href="https://docs.aws.amazon.com/general/latest/gr/aws-access-keys-best-practices.html">Best Practices for Managing AWS Access Keys </a>

## Configuring the credentials file
- Install the AWS CLI
    - `pip install awscli`
- After installing run the following:
    - `aws configure`
    - Copy and paste your Access Key ID and Secret Access Key when prompted

## Keeping EC2 Instances Secure
- The first step is to create an ssh key pair 
    - Amazon holds the public key, you hold the private key

- Any time you create an instance in the cloud, the default is that anyone can ssh into
    - They just need your private ssh key and the IP of your server
        - Ok, the chances of hackers acquiring both is small but it's still worth mitigating!
        
- Security Groups
    - Security groups allow you restrict access to your EC2 instance

## Creating a key pair
- Login to the <a href="https://aws.amazon.com/console/">AWS Management Console</a>
- Click the services tab in the upper left corner and search for EC2
- On the left side there is a tool bar
    - Click on "Key Pairs" underneath "NETWORK & SECURITY"
- Create a new pair and download the private key

<img src="./ec2_console_keypair.png" width="75%" height="75%"/>

## Launching an EC2 Instance
- Login to the <a href="https://aws.amazon.com/console/">AWS Management Console</a>
- Click the services tab in the upper left corner and search for EC2
- On the homepage for the EC2 service, click the "Launch Instance" button
    - Step 1) Selecting the Amazon Machine Image (AMI), i.e. the OS of your server
        - Amazon Linux 2 AMI (HVM), SSD Volume Type
    - Step 2) Selecting the Instance Type, i.e. the physical server infrastructure (CPUs, memory, etc..)
        - t2.micro
    - Step 3) Configuring the instance details
        - Leave the defaults
    - Step 4) Adding storage 
        - Increase the storage from 8 to 16 GiB
    - Step 5) Adding tags
        - Skip
    - Step 6) Configuring the Security Group
        - Create a new security group
        - Change the name and descriptiong to something useful
        - Update the "Source" to be "My IP" 
    - Step Launch
        - Select the key pair you created in the previous step



## Connecting to your instance
- After launching your instance, navigate to your running instances in the toolbar
    - Click on "Instances" undernearth "INSTANCES"
- Select your running instance and then click "connect"
    - Copy the example ssh command, it should look like this:
    
``` console
ssh -i "keyname" ec2-user@ec2_public_dns
```

- The keyname should be the name of the key you downloaded
- `ec2-user` should always be username
    - If you ever see "root" that means you are using your root-access account, which is not recommended 
- `ec2_public_dns` 
    - Each instance is assigned a public DNS hostname that resolves to its a unique IP address

    

## Setting up Miniconda3 
- Download the latest version of Miniconda3 for Linux
```shell
curl -OL https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
```
    
- Execute the installation script
```shell
bash ./Miniconda3-latest-Linux-x86_64.sh
```

## Copying over AWS access keys
- Remember that in order to access any AWS programmatically, we need to have our credentials configured properly
- In this case, we need to copy our credentials file to the EC2 instance
    - This can be done using either `rsync` or `scp`
- The command below will copy the entire `.aws` directory from your computer to your EC2 instance

```console
scp -ri "/path/to/aws_ssh_key.pem" ~/.aws/ ec2-user@$ec2_public_dns:~
```

- Update the command with the path to your ssh key and the DNS of your EC2 instance

## Install `astroquery` and Download data
- Install `astroquery` via `pip`
    - `pip install astroquery`

- Downloading data
    - Download by FLC and FLT images for HST Proposal ID, 14507, ACS/WFC External CTE Monitor
    
    
```python
from astroquery.mast import Observations
# Submit the query to MAST
obstable = Observations.query_criteria(
    proposal_id=14507, 
    obs_collection='HST'
)
# Get a table of all the products returned by the query
products = Observations.get_product_list(obstable)
# Filter those products to get the FLT and FLC images
filtered_products = Observations.filter_products(
    products,
    mrp_only=False, 
    productSubGroupDescription=['FLT', 'FLC']
)
# Download the filtered products
Observations.download_products(
    filtered_products['obsID'].tolist(), 
    mrp_only=False, 
    productSubGroupDescription=['FLT','FLC']
)
```

## Now we enable AWS downloads
- By default, `astroquery` will download from MAST
- To enable downloads from `S3` we use the `enable_cloud_dataset()` method of the `Observations` class
    - In the example below, I specify a particular profile within my AWS credentials file

```python
from astroquery.mast import Observations

Observations.enable_cloud_dataset('AWS', profile='ndmiles_admin')

# Submit the query to MAST
obstable = Observations.query_criteria(
    proposal_id=14507, 
    obs_collection='HST'
)
# Get a table of all the products returned by the query
products = Observations.get_product_list(obstable)
# Filter those products to get the FLT and FLC images
filtered_products = Observations.filter_products(
    products,
    mrp_only=False, 
    productSubGroupDescription=['FLT', 'FLC']
)
# Download the filtered products
Observations.download_products(
    filtered_products['obsID'].tolist(), 
    mrp_only=False, 
    productSubGroupDescription=['FLT','FLC']
)
```